In [1]:
import cortex
import numpy as np

from utils.utils import CV_ind
from utils.global_params import n_folds

Device:  cpu


In [2]:
surfaces = dict( F = 'fMRI_story_F',
	G = 'fMRI_story_G',
	H = 'fMRI_story_H',
	I = 'fMRI_story_I',
	J = 'fMRI_story_J',
	K = 'fMRI_story_K',
	L = 'fMRI_story_L',
	M = 'fMRI_story_M',
	N = 'fMRI_story_N')

transforms = dict( F = 'F_ars_auto2',
	G = 'G_ars_auto2',
	H = 'H_ars_auto2',
	I = 'I_ars_auto2',
	J = 'J_ars_auto2',
	K = 'K_ars_auto2',
	L = 'L_ars_auto2',
	M = 'M_ars_auto2',
	N = 'N_ars_auto2')

vols = {}

In [3]:
def roi_to_allvoxels(rois, roi_array):
    num_folds = roi_array.shape[0]
    num_voxels = rois.item()['F']['all'].shape[0]
    allvoxel_array = np.zeros((num_folds, num_voxels))
    allvoxel_array[:] = np.nan
    roi_ids = np.where(rois.item()['F']['all'] == 1)
    for i in range(num_folds):
        allvoxel_array[i, roi_ids] = roi_array[i]
    return allvoxel_array

In [4]:
sub = 'F'
model = 'mlp_allvoxels'
mask = cortex.db.get_mask(surfaces[sub], transforms[sub], 'thin')
rois = np.load('../HP_subj_roi_inds.npy', allow_pickle=True)

## Load Relevant Correlation and/or Accuracy Data

In [5]:
neighborhood_path = ''
noneighborhood_path = ''
encoder_output_path = ''

# neighborhood_loaded = np.load(neighborhood_path, allow_pickle=True)
# noneighborhood_loaded = np.load(noneighborhood_path, allow_pickle=True)
# output_loaded = np.load(encoder_output_path, allow_pickle=True).item()
# preds_loaded = output_loaded['preds_t']
# corrs_loaded = output_loaded['corrs_t']

If using only ROI correlations/accuracies, extend array size with nans for all other voxels:

In [89]:
# noneighborhood_loaded = roi_to_allvoxels(rois, noneighborhood_loaded)
# corrs_loaded = roi_to_allvoxels(rois, corrs_loaded)

In [82]:
# vols[sub + '_' + model + '_neighborhood'] = cortex.Volume(neighborhood_loaded.mean(0), surfaces[sub], transforms[sub], mask=mask, vmin=0.5, vmax=0.75, cmap='viridis')
# vols[sub + '_' + model + '_no_neighborhood'] = cortex.Volume(noneighborhood_loaded.mean(0), surfaces[sub], transforms[sub], mask=mask, vmin=0.5, vmax=0.75, cmap='viridis')
# vols[sub + '_' + model + '_corrs'] = cortex.Volume(corrs_loaded.mean(0), surfaces[sub], transforms[sub], mask=mask, vmin=0, vmax=0.2, cmap='viridis')


In [9]:
cortex.webshow(vols)

Started server on port 47578


<JS: window.viewer>

In [90]:
### PLOTTING STD DEVIATIONS ####

In [6]:
def fold_voxels_stderr(ind_num, n_folds, preds):
    n = preds.shape[0]
    ind = CV_ind(n, n_folds)
    preds_ind = ind==ind_num
    fold_preds = preds[preds_ind]
    print(np.std(fold_preds, 0))
    where_stderr_zero = (np.std(fold_preds, 0) != 0)
    return where_stderr_zero

# lin_folds = []
# for f in range(n_folds):
#     lin_folds.append(fold_voxels_stderr(f, n_folds, lin_preds_loaded))

# mlp_folds = []
# for f in range(n_folds):
#     mlp_folds.append(fold_voxels_stderr(f, n_folds, mlp_preds_loaded))
    
mlp_allvoxels_folds = []
for f in range(n_folds):
    mlp_allvoxels_folds.append(fold_voxels_stderr(f, n_folds, mlp_allvoxels_preds_loaded))

[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0.0036257  0.00298019 0.00833588 ... 0.00410301 0.00517578 0.00594044]


In [7]:
# for f in range(n_folds):
#     key_sfx = '_lin_fold{}_stderr'.format(f)
#     key = sub+key_sfx
#     vols[key] = cortex.Volume(lin_folds[f], surfaces[sub], transforms[sub], mask=mask, vmin=0, vmax=1, cmap='viridis')
    
# for f in range(n_folds):
#     key_sfx = '_mlp_fold{}_stderr'.format(f)
#     key = sub+key_sfx
#     vols[key] = cortex.Volume(mlp_folds[f], surfaces[sub], transforms[sub], mask=mask, vmin=0, vmax=1, cmap='viridis')

for f in range(n_folds):
    key_sfx = '_mlp_allvoxels_fold{}_stderr'.format(f)
    key = sub+key_sfx
    vols[key] = cortex.Volume(mlp_allvoxels_folds[f], surfaces[sub], transforms[sub], mask=mask, vmin=0, vmax=1, cmap='viridis')

In [ ]:
### END PLOTTING STD DEVIATIONS ####